In [ ]:
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#
#from selenium.webdriver.common.action_chains import ActionChains
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#import pickle
#
#import os
#import time
#import pandas as pd
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#
#from selenium import webdriver
#from selenium.webdriver.firefox.options import Options
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#import time
#import pandas as pd
#
#from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

In [11]:
import pandas as pd
import os
import pickle
import time
import json
from collections import defaultdict
from openai import OpenAI
from datetime import datetime
import subprocess
import re

api_key='sk-bedceae2ceba437f944db22706354095'


In [ ]:
smile2jannahURLs=[
'https://www.youtube.com/watch?v=1suta8hYDxo',
'https://www.youtube.com/watch?v=Ojm4C0qwhMM',
'https://www.youtube.com/watch?v=ItK0890LP-g',
'https://www.youtube.com/watch?v=dIAMoGzeut0',
'https://www.youtube.com/watch?v=feMoy5ioaeU',
'https://www.youtube.com/watch?v=Slx5ZH5NZBM',
'https://www.youtube.com/watch?v=aGkNouIMzFY',
'https://www.youtube.com/watch?v=4XaMJrS9DvY',
'https://www.youtube.com/watch?v=6xhUFLlG_58',
'https://www.youtube.com/watch?v=x84vLClrXHw',
'https://www.youtube.com/watch?v=YpZnd1xmP_4',
'https://www.youtube.com/watch?v=irE6mn-dDZk',
'https://www.youtube.com/watch?v=RPd-Buti7gk',
'https://www.youtube.com/watch?v=xCfrRSPtH58',
    
]

smile2jannahIDs=[url.split('/watch?v=')[-1] for url in smile2jannahURLs]


In [ ]:
smile2jannahIDs

In [ ]:

#https://www.youtube.com/watch?v=dCHh2HacOYk
# Trump MELTDOWN on Walmart + Saudi news
# 0UtHedAXIkA
# Indians Finally Admit Pakistan Won

# https://www.youtube.com/watch?v=xJ_omX4Ejqw
# India Beat Pakistan's Air Defense

# https://www.youtube.com/watch?v=7tqYgfhCwb4&t=154s&pp=ygUOc21pbGUgMiBqYW5uYWg%3D
# Indians Declare WAR on Aamir Khan


### below is to run a loop and get all

In [ ]:


for vide_num,video_id in enumerate(smile2jannahIDs):  
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    print(vide_num,'/',len(smile2jannahIDs))
    video_url = f"https://www.youtube.com/watch?v={video_id}"
    
    
    command = [
        "yt-dlp",
        "--skip-download",
        "--write-comments",
        "--no-warnings",
        "--output", f"comments_scraped/{video_id}_at_{timestamp}",
        video_url
    ]
    
    try:
        result = subprocess.run(command, check=True, text=True, capture_output=True)
        print("yt-dlp ran successfully.")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print("yt-dlp failed.")
        print(e.stderr)

### below is the loop to process all files in the comment_scraped folder

In [2]:
comment_files=os.listdir('comments_scraped/')
print(comment_files)

['4XaMJrS9DvY_at_20250519_191428.info.json', 'aGkNouIMzFY_at_20250519_191428.info.json', 'xCfrRSPtH58_at_20250519_191628.info.json', 'YpZnd1xmP_4_at_20250519_191428.info.json', 'x84vLClrXHw_at_20250519_191428.info.json', 'irE6mn-dDZk_at_20250519_191428.info.json', '.DS_Store', '1suta8hYDxo_at_20250519_191428.info.json', '6xhUFLlG_58_at_20250519_191428.info.json', 'dIAMoGzeut0_at_20250519_191428.info.json', 'ItK0890LP-g_at_20250519_191428.info.json', 'irE6mn-dDZk_at_20250519_191928.info.json', 'RPd-Buti7gk_at_20250519_191428.info.json', 'Slx5ZH5NZBM_at_20250519_191428.info.json', 'Ojm4C0qwhMM_at_20250519_191428.info.json', 'xCfrRSPtH58_at_20250519_191428.info.json', 'feMoy5ioaeU_at_20250519_191428.info.json']


In [3]:
# Remove system files
comment_files = [f for f in comment_files if f.endswith(".info.json")]

# Dictionary to hold latest file per video ID
latest_files = {}

for filename in comment_files:
    # Extract video ID and timestamp
    match = re.match(r"(.+?)_at_(\d+)_\d+\.info\.json", filename)
    if match:
        video_id = match.group(1)
        timestamp_str = match.group(2)
        timestamp = datetime.strptime(timestamp_str, "%Y%m%d")

        if video_id not in latest_files or timestamp > latest_files[video_id][0]:
            latest_files[video_id] = (timestamp, filename)

# Final list of filenames to process
selected_files = [tup[1] for tup in latest_files.values()]

print("Selected files:", selected_files)

Selected files: ['4XaMJrS9DvY_at_20250519_191428.info.json', 'aGkNouIMzFY_at_20250519_191428.info.json', 'xCfrRSPtH58_at_20250519_191628.info.json', 'YpZnd1xmP_4_at_20250519_191428.info.json', 'x84vLClrXHw_at_20250519_191428.info.json', 'irE6mn-dDZk_at_20250519_191428.info.json', '1suta8hYDxo_at_20250519_191428.info.json', '6xhUFLlG_58_at_20250519_191428.info.json', 'dIAMoGzeut0_at_20250519_191428.info.json', 'ItK0890LP-g_at_20250519_191428.info.json', 'RPd-Buti7gk_at_20250519_191428.info.json', 'Slx5ZH5NZBM_at_20250519_191428.info.json', 'Ojm4C0qwhMM_at_20250519_191428.info.json', 'feMoy5ioaeU_at_20250519_191428.info.json']


In [12]:
for vide_num,filename in enumerate(selected_files):  
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    match = re.match(r"(.+?)_at_(\d+)_\d+\.info\.json", filename)
    

    if match:
        print('matched')
        video_id = match.group(1)
    print(filename,video_id)

    # Load your JSON file
    with open(f"comments_scraped/{filename}", "r", encoding="utf-8") as f:
        all_data=json.load(f)        
        raw_comments = all_data['comments']
        title=all_data['title']
        uploader_id=all_data['uploader_id']
    
    print(title,uploader_id)
    
    # Group comments by ID and organize replies under their parent
    comment_dict = {}
    replies_grouped = defaultdict(list)
    
    # First pass to separate top-level and replies
    for comment in raw_comments:
        
        if comment.get("parent") and comment["parent"] != "root":
            replies_grouped[comment["parent"]].append({
                "id": comment["id"],
                "text": comment["text"],
                "author": comment["author"],
                "likes": comment["like_count"],
                "timestamp": comment["timestamp"]
            })
        else:
            comment_dict[comment["id"]] = {
                "id": comment["id"],
                "text": comment["text"],
                "author": comment["author"],
                "likes": comment["like_count"],
                "timestamp": comment["timestamp"],
                "replies": []
            }
    
    # Second pass to attach replies to top-level comments
    for parent_id, replies in replies_grouped.items():
        if parent_id in comment_dict:
            comment_dict[parent_id]["replies"] = replies
    
    # Convert dict to list
    structured_comments = list(comment_dict.values())
    
        
    base_filename=f'{video_id}'
    rows = []
    for comment in structured_comments:
        rows.append({
            "id": comment["id"],
            "text": comment["text"],
            "author": comment["author"],
            "likes": comment["likes"],
            "timestamp": comment["timestamp"],
            "is_reply": False,
            "parent_id": None
        })
        for reply in comment.get("replies", []):
            rows.append({
                "id": reply["id"],
                "text": reply["text"],
                "author": reply["author"],
                "likes": reply["likes"],
                "timestamp": reply["timestamp"],
                "is_reply": True,
                "parent_id": comment["id"]
            })
    
    # Create a DataFrame
    df = pd.DataFrame(rows)
    print(df.shape)
    df.sort_values(by=['likes'],inplace=True,ascending=False)

    # Export to CSV
    df.to_csv(f"comments_scraped_csv/{base_filename}.csv", index=False)
    
    # Export to Pickle
    with open(f"comments_scraped_pkl/{base_filename}.pkl", "wb") as f:
        pickle.dump(structured_comments, f)
    
    #print("Export completed: CSV and PKL saved.")
    
    df_comment1=df[df['is_reply']==False]

    comment1_list=list(df_comment1['text'])
    k=5
    topkcomments_list=comment1_list[:int(0.05*len(comment1_list))]
    
    topkcomments_str='\n'.join(topkcomments_list)


    client = OpenAI(
        api_key=api_key,
        base_url="https://api.deepseek.com",
    )
    
    system_prompt = """
    The user will provide a list of YouTube comments. Please analyze all the comments together and generate a single, structured JSON object summarizing the overall qualitative dynamics.
    
    EXAMPLE INPUT:
    ["Great job! This video really opened my eyes.", "What a biased take. Shameful.", "😂😂 you're so clueless it's funny.", "Pakistan zindabad!", "Link to giveaway 👉 http://spamlink"]
    
    EXAMPLE JSON OUTPUT:
    {
      "overall_sentiment_distribution": {
        "positive": 1,
        "neutral": 1,
        "negative": 3
      },
      "dominant_emotions": ["anger", "sarcasm", "joy"],
      "toxic_comment_count": 2,
      "controversy_score": 0.75,
      "key_topics": ["bias in media", "nationalism", "truth and misinformation"],
      "frequent_bias_or_group_mentions": ["Pakistan", "India", "YouTube creators"],
      "sarcasm_detected": true,
      "languages_detected": ["English", "Urdu"],
      "spam_comment_count": 1,
      "summary": "The comment section is emotionally charged with a mix of national pride, strong criticism, and sarcasm. There's significant polarization, and a moderate amount of toxicity and spam. These patterns suggest the topic is controversial and has drawn viewers from diverse backgrounds with opposing views."
    }
    
    Please return the analysis in this format. Focus on producing insightful, unbiased feedback that helps a content creator understand audience sentiment, engagement, controversy, and risks.
    """
    
    user_prompt = topkcomments_str
    
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        response_format={
            'type': 'json_object'
        }
    )
    
    print(json.loads(response.choices[0].message.content))
    
    llm_analysis=json.loads(response.choices[0].message.content)
    llm_analysis['title']=title
    llm_analysis['uploader_id']=uploader_id
    
    
    
    # Construct the filename
    filename = f"insights/{video_id}_insight1_{timestamp}.json"
    # Save to JSON file
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(llm_analysis, f, ensure_ascii=False, indent=2)
    print(f"Saved analysis to {filename}")    


    print('*'*10)
    
    

matched
4XaMJrS9DvY_at_20250519_191428.info.json 4XaMJrS9DvY
Pakistan Just Humiliated India on the World Stage @smiletojannah
(3034, 7)
{'overall_sentiment_distribution': {'positive': 35, 'neutral': 5, 'negative': 10}, 'dominant_emotions': ['pride', 'solidarity', 'joy', 'sarcasm'], 'toxic_comment_count': 3, 'controversy_score': 0.6, 'key_topics': ['Pakistani military prowess', 'Muslim unity', 'India-Pakistan relations', 'support for Palestine and Kashmir'], 'frequent_bias_or_group_mentions': ['Pakistan', 'India', 'Muslim Ummah', 'Turkey', 'China', 'Bangladesh'], 'sarcasm_detected': True, 'languages_detected': ['English', 'Urdu', 'Arabic'], 'spam_comment_count': 1, 'summary': 'The comment section is overwhelmingly positive, reflecting strong feelings of pride and solidarity among Muslims worldwide, particularly in support of Pakistan. There is a notable emphasis on Pakistani military achievements, Muslim unity, and criticism of India. The presence of sarcasm and a few toxic comments ind

In [9]:
df

,id,text,author,likes,timestamp,is_reply,parent_id
4632,Ugyjm_ZFvJ-zcbu4T-l4AaABAg,We are Muslims I'm from Maldives standing with...,@ibrahimthalal1262,3400,1747180800,False,None
4451,UgzBxgAPnu2mEyX5TZF4AaABAg,"I’m Afghan, Pakistan all the way it is nifaaq ...",@KandahariHadithLover,2900,1747180800,False,None
2693,UgxqVeyQUkZJFVF5g354AaABAg,I used to be a hindu then became an atheist bu...,@DhruvS89,2400,1747180800,False,None
3832,UgxkxKJzuZy6lHq1y5R4AaABAg,We stand with PAKISTAN salam from Indonesia,@dewirosalina9774,2300,1747180800,False,None
5700,Ugx3JAcboqt0nnC2x5x4AaABAg,I am a Bangladeshi and believe me we are not w...,@RoronoaZoro-mw7wu,1800,1747180800,False,None
...,...,...,...,...,...,...,...
2322,UgwDEUhU_daLsphejhB4AaABAg,Chaman chutia nation 🇮🇳,@nitrobot3206,0,1747180800,False,None
2321,UgyvYANkiUpcErZSy_J4AaABAg,Modi emerge and he's appears understand that!!😂😂😂,@Mr.Lan_JiAooo01,0,1747180800,False,None
2320,Ugzq_Wyw1YiqAPeBwl14AaABAg,Hats off to you man ❤❤❤,@MuhammadUsman-ks4ng,0,1747180800,False,None
2319,Ugw9093V8SovRpip7np4AaABAg,This is like the Greater Israel project.,@DZULKAFLEYABDSAMAD-s4r,0,1747180800,False,None


In [ ]:
df_comment1

In [ ]:
print(comment1_list[:2])

In [ ]:
len(topkcomments_list),len(comment1_list)